In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
!pip install emoji
!pip install tensorflow
!pip install transformers

     |████████████████████████████████| 175 kB 9.6 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=23585b15169f30eeee59849eda8fbb44d794eac03e24288f81bd127d9bb32796
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 462 kB 20.6 MB/s 
     |████████████████████████████████| 4.2 MB 10.0 MB/s 
     |████████████████████████████████| 84 kB 1.6 MB/s 
     |████████████████████████████████| 6.6 MB 19.9 MB/s 
     |████████████████████████████████| 596 kB 36.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained('rabindralamsal/BERTsent')

model = TFAutoModelForSequenceClassification.from_pretrained('rabindralamsal/BERTsent')

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/BERTsent.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
link = 'https://drive.google.com/file/d/1a1mZHqe3o1psMlsaXrqIUOe9IiYQ2gRT/view?usp=sharing'

In [5]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Survey_Results_Labeled_Dec.csv') 
 
df = pd.read_csv('Survey_Results_Labeled_Dec.csv')
df


,Unnamed: 0,text,1,2,3,Label
0,Q1,"""Tucker Carlson seem to have forgotten that Ru...",0.1,0.0,0.9,negative
1,Q2,"""Analysis: Would Putin get away with invading ...",0.0,0.4,0.6,negative
2,Q3,"""Kiev, Ukraine - Ukrainian defense officials s...",0.0,0.8,0.2,neutral
3,Q4,Russia-Ukraine tensions: why Russia is putting...,0.0,0.5,0.5,neutral
4,Q5,@ManMadeMoon I don’t think it’s that deep. I t...,0.1,0.0,0.9,negative
...,...,...,...,...,...,...
97,Q96,"""@peterpomeranzev Putin doesn't want to invade...",0.0,0.2,0.8,negative
98,Q97,@JonScottFNC Ukrainians remember The Holodomor...,0.0,0.2,0.8,negative
99,Q98,‘Fake news’ may complicate Putin-Biden talks –...,0.0,0.9,0.1,neutral
100,Q99,"""Russia kept EU busy by instigating a hybrid w...",0.0,0.5,0.5,neutral


In [6]:
df['text'] = df['text'].astype('str') 

In [7]:
docs = list(df.loc[:, "text"].values)

In [8]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"BERTsent Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 102/102 [01:07<00:00,  1.52it/s]


In [9]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)

In [10]:
df

,Unnamed: 0,text,1,2,3,Label,BERTsent Label,Negative,Neutral,Positive
0,Q1,"""Tucker Carlson seem to have forgotten that Ru...",0.1,0.0,0.9,negative,1,0.474342,0.519012,0.006646
1,Q2,"""Analysis: Would Putin get away with invading ...",0.0,0.4,0.6,negative,1,0.131044,0.860338,0.008618
2,Q3,"""Kiev, Ukraine - Ukrainian defense officials s...",0.0,0.8,0.2,neutral,1,0.018560,0.969049,0.012391
3,Q4,Russia-Ukraine tensions: why Russia is putting...,0.0,0.5,0.5,neutral,1,0.179818,0.812392,0.007791
4,Q5,@ManMadeMoon I don’t think it’s that deep. I t...,0.1,0.0,0.9,negative,0,0.963008,0.033810,0.003183
...,...,...,...,...,...,...,...,...,...,...
97,Q96,"""@peterpomeranzev Putin doesn't want to invade...",0.0,0.2,0.8,negative,0,0.838630,0.158122,0.003249
98,Q97,@JonScottFNC Ukrainians remember The Holodomor...,0.0,0.2,0.8,negative,0,0.877747,0.116980,0.005273
99,Q98,‘Fake news’ may complicate Putin-Biden talks –...,0.0,0.9,0.1,neutral,0,0.699853,0.295748,0.004398
100,Q99,"""Russia kept EU busy by instigating a hybrid w...",0.0,0.5,0.5,neutral,1,0.427984,0.565815,0.006201


In [11]:
#delete control tweets from set
df = df.drop(33)
df = df.drop(67)

In [12]:
#change column names to make clear what is what
df = df.rename(columns={"Label":"Manual Label"})

In [13]:
df

,Unnamed: 0,text,1,2,3,Manual Label,BERTsent Label,Negative,Neutral,Positive
0,Q1,"""Tucker Carlson seem to have forgotten that Ru...",0.1,0.0,0.9,negative,1,0.474342,0.519012,0.006646
1,Q2,"""Analysis: Would Putin get away with invading ...",0.0,0.4,0.6,negative,1,0.131044,0.860338,0.008618
2,Q3,"""Kiev, Ukraine - Ukrainian defense officials s...",0.0,0.8,0.2,neutral,1,0.018560,0.969049,0.012391
3,Q4,Russia-Ukraine tensions: why Russia is putting...,0.0,0.5,0.5,neutral,1,0.179818,0.812392,0.007791
4,Q5,@ManMadeMoon I don’t think it’s that deep. I t...,0.1,0.0,0.9,negative,0,0.963008,0.033810,0.003183
...,...,...,...,...,...,...,...,...,...,...
97,Q96,"""@peterpomeranzev Putin doesn't want to invade...",0.0,0.2,0.8,negative,0,0.838630,0.158122,0.003249
98,Q97,@JonScottFNC Ukrainians remember The Holodomor...,0.0,0.2,0.8,negative,0,0.877747,0.116980,0.005273
99,Q98,‘Fake news’ may complicate Putin-Biden talks –...,0.0,0.9,0.1,neutral,0,0.699853,0.295748,0.004398
100,Q99,"""Russia kept EU busy by instigating a hybrid w...",0.0,0.5,0.5,neutral,1,0.427984,0.565815,0.006201


In [14]:
df['BERTsent Label'].replace({0: "negative", 1: "neutral", 2: 'positive'}, inplace=True)

In [20]:
 import sklearn
 from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support
 b_a_Vader = balanced_accuracy_score(df['Manual Label'], df['BERTsent Label'])
 p_r_f1_class_Vader = precision_recall_fscore_support(df['Manual Label'], df['BERTsent Label'], average=None, labels=['negative', 'neutral', 'positive'])
 p_r_f1_Vader = precision_recall_fscore_support(df['Manual Label'], df['BERTsent Label'], average='macro')

In [21]:
print(b_a_Vader)
print(p_r_f1_Vader)
print(p_r_f1_class_Vader)

0.6125356125356125
(0.8366753850624818, 0.6125356125356125, 0.655588224999125, None)
(array([0.86486486, 0.64516129, 1.        ]), array([0.61538462, 0.88888889, 0.33333333]), array([0.71910112, 0.74766355, 0.5       ]), array([52, 45,  3]))


In [ ]:
import pandas as pd
y_actu = df['Manual Label']
y_pred = df['BERTsent Label']
df_confusion = pd.crosstab(y_actu, y_pred)

In [ ]:
df_confusion

BERTsent Label,negative,neutral,positive
Manual Label,,,
negative,32,20,0
neutral,5,40,0
positive,0,2,1


In [ ]:
#Get accuracies per class order (negative, neutral, positive)
from sklearn.metrics import confusion_matrix
y_true = df['Manual Label']
y_pred = df['BERTsent Label']
matrix = confusion_matrix(y_true, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.61538462, 0.88888889, 0.33333333])

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
df.to_csv('BERTsent_Test_Dec.csv')
!cp BERTsent_Test_Dec.csv "/content/drive/My Drive/Colab Notebooks"

